In [1]:
%matplotlib inline
import sys
sys.path.append("../") # go to parent dir

import numpy as np
import torch
import matplotlib.pyplot as plt
import mpmath

from synthetic.generate import SingleTaskTreeDepsGenerator
from metal.label_model import LabelModel
from metal.label_model.utils import (
    compute_mu,
    compute_covariance,
    compute_inv_covariance,
    print_matrix,
    visualize_matrix
)

/lfs/1/annhe/anaconda3/envs/metal/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Generate synthetic dataset using the deterministic edge function
np.random.seed(1)
N = 50
M = 5
K = 2
edges_list = []
for i in range(M):
    edges_list.append((i,i))
edges_list.append((1,2))
edges_list.append((2,1))
data = SingleTaskTreeDepsGenerator(N, M, k=K, edges_list=edges_list)


BANANA BANANA
HELLO HELLO

(0, 1)   [0.4753198  0.74829204]
(0, 2)   [0.10010294 0.37209932]
(1, 1)   [0.2320803  0.18310474]
(1, 2)   [0.26763419 0.41100465]
(2, 1)   [0.45709073 0.58493506]
(2, 2)   [0.47727506 0.71669755]
(3, 1)   [0.28400702 0.89030569]
(3, 2)   [0.12464883 0.70342076]
(4, 1)   [0.47557432 0.60282085]
(4, 2)   [0.22634824 0.27829134]
((0, 0), 1, 1)   [0.82067011 0.97143542]
((0, 0), 1, 2)   [0.38208176 0.72309035]
((0, 0), 2, 1)   [0.88875024 0.905146  ]
((0, 0), 2, 2)   [0.17653979 0.1351493 ]
((1, 1), 1, 1)   [0.25284738 0.89032825]
((1, 1), 1, 2)   [0.18851215 0.47899686]
((1, 1), 2, 1)   [0.96210058 0.57984876]
((1, 1), 2, 2)   [0.7226894  0.38396407]
((2, 2), 1, 1)   [0.71785083 0.8511631 ]
((2, 2), 1, 2)   [0.11645945 0.77512988]
((2, 2), 2, 1)   [0.98997498 0.77334909]
((2, 2), 2, 2)   [0.35239959 0.8103514 ]
((3, 3), 1, 1)   [0.19290341 0.50310417]
((3, 3), 1, 2)   [0.91773595 0.36425273]
((3, 3), 2, 1)   [0.3589978  0.21702571]
((3, 3), 2, 2)   [0.11743026

P(L_ 0 = 0 , L_ 4 = 1  | Y =  2 ) =  0.219098154294588
Trying to retrieve for i:  0
P(L_ 0 = 0 , L_ 4 = 2  | Y =  2 ) =  0.219098154294588
Trying to retrieve for i:  0
P(L_ 0 = 1 , L_ 4 = 0  | Y =  2 ) =  0.4630392698738982
Trying to retrieve for i:  0
P(L_ 0 = 1 , L_ 4 = 1  | Y =  2 ) =  0.4630392698738982
Trying to retrieve for i:  0
P(L_ 0 = 1 , L_ 4 = 2  | Y =  2 ) =  0.4630392698738982
Trying to retrieve for i:  0
P(L_ 0 = 2 , L_ 4 = 0  | Y =  2 ) =  0.3178625758315137
Trying to retrieve for i:  0
P(L_ 0 = 2 , L_ 4 = 1  | Y =  2 ) =  0.3178625758315137
Trying to retrieve for i:  0
P(L_ 0 = 2 , L_ 4 = 2  | Y =  2 ) =  0.3178625758315137
Labelers =  (1, 2)
Trying to retrieve for i:  1
P(L_ 1 = 0 , L_ 2 = 0  | Y =  2 ) =  0.219098154294588
Trying to retrieve for i:  1
P(L_ 1 = 0 , L_ 2 = 1  | Y =  2 ) =  0.39325367344349443
Trying to retrieve for i:  1
P(L_ 1 = 0 , L_ 2 = 2  | Y =  2 ) =  0.4486384694604528
Trying to retrieve for i:  1
P(L_ 1 = 1 , L_ 2 = 0  | Y =  2 ) =  0.263123777

Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  2
Trying to retr

Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retr

Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retr

Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retr

Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retr

Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retr

In [3]:
# Generate synthetic dataset
np.random.seed(1)
N = 50
M = 5
K = 2
EDGE_PROB=1.0
data = SingleTaskTreeDepsGenerator(N, M, k=K, edge_prob=EDGE_PROB)

BANANA BANANA
(0, 1)   [0.44911967 0.70277143]
(0, 2)   [0.94198516 0.86167983]
(1, 1)   [0.38194617 0.57209334]
(1, 2)   [0.4991076  0.30661949]
(2, 1)   [0.58097252 0.92256582]
(2, 2)   [0.51148433 0.48762871]
(3, 1)   [0.94521501 0.80055031]
(3, 2)   [0.74437346 0.82248175]
(4, 1)   [0.18352073 0.56633729]
(4, 2)   [0.87851823 0.84623222]
((0, 1), 1, 1)   [0.84664302 0.34574498]
((0, 1), 1, 2)   [0.15331888 0.70347524]
((0, 1), 2, 1)   [0.63375897 0.70448869]
((0, 1), 2, 2)   [0.47060909 0.27779581]
((1, 2), 1, 1)   [0.36066668 0.22790812]
((1, 2), 1, 2)   [0.80498303 0.47128496]
((1, 2), 2, 1)   [0.13075418 0.66162699]
((1, 2), 2, 2)   [0.69457216 0.36864577]
((0, 3), 1, 1)   [0.50152106 0.29991209]
((0, 3), 1, 2)   [0.16602775 0.52231467]
((0, 3), 2, 1)   [0.18655503 0.91303315]
((0, 3), 2, 2)   [0.20754142 0.57231944]
((0, 4), 1, 1)   [0.1752607 0.9251752]
((0, 4), 1, 2)   [0.91940354 0.3690371 ]
((0, 4), 2, 1)   [0.62595021 0.60932082]
((0, 4), 2, 2)   [0.65254449 0.96088209]
Tr

P(L_ 1 = 1 , L_ 3 = 2  | Y =  1 ) =  0.18778873501309776
Trying to retrieve for i:  1
P(L_ 1 = 2 , L_ 3 = 0  | Y =  1 ) =  0.05223513988605153
Trying to retrieve for i:  1
P(L_ 1 = 2 , L_ 3 = 1  | Y =  1 ) =  0.17726845172154496
Trying to retrieve for i:  1
P(L_ 1 = 2 , L_ 3 = 2  | Y =  1 ) =  0.1314683039810967
Labelers =  (1, 4)
Trying to retrieve for i:  1
P(L_ 1 = 0 , L_ 4 = 0  | Y =  1 ) =  0.014782476418998481
Trying to retrieve for i:  1
P(L_ 1 = 0 , L_ 4 = 1  | Y =  1 ) =  0.02652557465278036
Trying to retrieve for i:  1
P(L_ 1 = 0 , L_ 4 = 2  | Y =  1 ) =  0.07026648070538415
Trying to retrieve for i:  1
P(L_ 1 = 1 , L_ 4 = 0  | Y =  1 ) =  0.06722779946206554
Trying to retrieve for i:  1
P(L_ 1 = 1 , L_ 4 = 1  | Y =  1 ) =  0.1216602001447608
Trying to retrieve for i:  1
P(L_ 1 = 1 , L_ 4 = 2  | Y =  1 ) =  0.3385655730273176
Trying to retrieve for i:  1
P(L_ 1 = 2 , L_ 4 = 0  | Y =  1 ) =  0.04714198727811232
Trying to retrieve for i:  1
P(L_ 1 = 2 , L_ 4 = 1  | Y =  1 ) =  

P(L_ 1 = 2 , L_ 4 = 1  | Y =  2 ) =  0.15689451521504583
Trying to retrieve for i:  1
P(L_ 1 = 2 , L_ 4 = 2  | Y =  2 ) =  0.19513506039131676
Labelers =  (2, 3)
Trying to retrieve for i:  2
P(L_ 2 = 0 , L_ 3 = 0  | Y =  2 ) =  0.015612210147746457
Trying to retrieve for i:  2
P(L_ 2 = 0 , L_ 3 = 1  | Y =  2 ) =  0.06752300221594582
Trying to retrieve for i:  2
P(L_ 2 = 0 , L_ 3 = 2  | Y =  2 ) =  0.05979418242295495
Trying to retrieve for i:  2
P(L_ 2 = 1 , L_ 3 = 0  | Y =  2 ) =  0.056939320770314585
Trying to retrieve for i:  2
P(L_ 2 = 1 , L_ 3 = 1  | Y =  2 ) =  0.24405935449036403
Trying to retrieve for i:  2
P(L_ 2 = 1 , L_ 3 = 2  | Y =  2 ) =  0.21803993732517532
Trying to retrieve for i:  2
P(L_ 2 = 2 , L_ 3 = 0  | Y =  2 ) =  0.03684232161046833
Trying to retrieve for i:  2
P(L_ 2 = 2 , L_ 3 = 1  | Y =  2 ) =  0.15991737733108752
Trying to retrieve for i:  2
P(L_ 2 = 2 , L_ 3 = 2  | Y =  2 ) =  0.14127229368594302
Labelers =  (2, 4)
Trying to retrieve for i:  2
P(L_ 2 = 0 , L

Trying to retrieve for i:  0
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  4
Trying to retr

Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  0
Trying to retrieve for i:  2
Trying to retr

Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  3
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  4
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  1
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  2
Trying to retrieve for i:  3
Trying to retr